In [1]:
import os
from openai import OpenAI
import openai
import json
from Preprocessing.feature_extraction import load_eeg_data
from train_utils import evaluate

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [3]:
def create_file(train_jsonl_dir, val_jsonl_dir):
    """
    Create a file in the OpenAI API
    After creating the file, you should see the file ID(train/test) in the website(platform.openai.com).
    :param train_jsonl_dir: A directory of the train data in jsonl format
    :param val_jsonl_dir: A directory of the test data in jsonl format
    :return:
    """
    # load train data
    client.files.create(
        file=open(train_jsonl_dir, 'rb'),
        purpose='fine-tune'
    )
    print(f"Loaded {train_jsonl_dir}")

    # load validation data
    client.files.create(
        file=open(val_jsonl_dir, 'rb'),
        purpose='fine-tune'
    )
    print(f"Loaded {val_jsonl_dir}")

In [4]:
base_path = '/Users/imdohyeon/Library/CloudStorage/GoogleDrive-dhlim1598@gmail.com/공유 드라이브/4N_PKNU/BXAI/EEG-LLM/Dataset/'
train_jsonl_dir = base_path + '4ch/jsonl/train.jsonl'
val_jsonl_dir = base_path + '4ch/jsonl/val.jsonl'

In [5]:
window_size = 1000
selected_columns = [13, 28, 31, 34]
test_csv = base_path + '4ch/test.csv'

In [6]:
create_file(train_jsonl_dir, val_jsonl_dir)

Loaded /Users/imdohyeon/Library/CloudStorage/GoogleDrive-dhlim1598@gmail.com/공유 드라이브/4N_PKNU/BXAI/EEG-LLM/Dataset/4ch/jsonl/train.jsonl
Loaded /Users/imdohyeon/Library/CloudStorage/GoogleDrive-dhlim1598@gmail.com/공유 드라이브/4N_PKNU/BXAI/EEG-LLM/Dataset/4ch/jsonl/val.jsonl


In [9]:
training_file = 'file-1zeQudsjvzs8MttoduHssI1z'  # Ensure this should be checked in the website(platform.openai.com)
val_file = 'file-RAiZek9pppZp5yaaRDwL4poB'  # Ensure this should be checked in the website(platform.openai.com)
model = 'gpt-3.5-turbo-1106'

In [10]:
def train(training_file, val_file, model):
    """
    Fine-tuning the GPT model.
    After training, you should check the name of the model in the website(platform.openai.com).
    :param training_file: The ID of the training file
    :param val_file: The ID of the validation file
    :param model: anything you want(davinci-002 / gpt-3.5-turbo / and so on)
    """
    # start fine-tuning
    client.fine_tuning.jobs.create(
        training_file=training_file,
        validation_file=val_file,
        model=model
        # default=auto, thus it doesn't need to be specified
        # hyperparameters={
        #     'n_epochs':10,
        #     'batch_size':16,
        #     'learning_rate_multiplier':1e-4
        # }
    )

    print("Fine-tuning ended.")

In [11]:
train(training_file, val_file, model)

Fine-tuning ended.


In [ ]:
# Evaluate the fine-tuned model
model_id = ''

data, label = load_eeg_data(test_csv)
evaluate(data, label, window_size, selected_columns, model_id)